# This pipeline is meant to be run on a kubernetes cluster only, Do not run on your local machine directly to avoid causing complications in your development environment

# Import the needed libraries

In [1]:
!python -m pip install --user --upgrade pip

!pip3 install spacy tensorflow==2.3.0, keras==2.2.4 pandas==0.24.2 matplotlib==3.2.2 scipy==1.4.1 statsmodels==0.12.0 scikit-learn==0.23.1 --user

#!python -m spacy download en_core_web_sm

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Invalid requirement: 'tensorflow==2.3.0,'


In [2]:
# Install or Upgrade if present the KFP library

!python3 -m pip install https://storage.googleapis.com/ml-pipeline/release/0.1.10/kfp.tar.gz --upgrade

!python3 -m pip install --upgrade kfp --user

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://storage.googleapis.com/ml-pipeline/release/0.1.10/kfp.tar.gz (129 kB)
  Using cached google_auth-1.6.1-py2.py3-none-any.whl (68 kB)
  Created wheel for kfp: filename=kfp-0.1-py3-none-any.whl size=211223 sha256=690972feaeec81ce850df81bc9ac1457bc695b130c3576d5a0138c5a23928a10
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ap_goj8/wheels/98/cf/56/0c692ab36d3472d515b7b973fab48add9214110e4bdcd0ca7f
Successfully built kfp
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.24.0
    Uninstalling google-auth-1.24.0:
      Successfully uninstalled google-auth-1.24.0
  Attempting uninstall: kfp
    Found existing installation: kfp 1.1.2
    Uninstalling kfp-1.1.2:
      Successfully uninstalled kfp-1.1.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not cu

In [3]:
#Check if the install was successful

!which dsl-compile

In [4]:
# Import Kubeflow SDK
import kfp
import kfp.dsl as dsl
import kfp.components as comp


# where the outputs are stored
data_path = "pipe_data"


# Model Insertion

In [5]:
def deploy(data_path):
    
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy==2.2.0'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==2.2.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz'])
    
    import pandas as pd
    import tensorflow as tf
    from IPython import get_ipython
    from math import sqrt
    from numpy import concatenate
    from pandas import read_csv, DataFrame, concat
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from sklearn.metrics import mean_squared_error
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.utils import to_categorical
    from keras import regularizers
    from keras.optimizers import SGD, Adam, RMSprop

    import pandas as pd
    import numpy as np
    import pickle
    import spacy
    
    try:
        # check if the model configuration files are present in the pipe_data directory, if it isnt we would build our model and add them 
        arc = os.path.join(os.getcwd(),'pipe_data/model.json')
        json_file = open(arc, 'r')
        
    except:
    
        data = pd.read_csv('https://raw.githubusercontent.com/Chizzy-codes/g02-sms-spam/master/data/spam.csv', usecols=['v1', 'v2'],  encoding = 'latin-1')
        data2 = pd.read_csv('https://raw.githubusercontent.com/Chizzy-codes/g02-sms-spam/master/data/spam_additional.csv', usecols=['Text', 'v3'],  encoding = 'latin-1')

        embedding_dim = 100
        max_length = 150
        trunc_type='post'
        padding_type='post'
        oov_tok = "<OOV>"


        data.dropna(inplace=True,axis=1)
        data.rename(columns={"v1":"label","v2":"message"},inplace=True)
        data2 = data2[['Text','v3']].copy()
        data2.rename(columns={"v3":"label","Text":"message"},inplace=True)

        # merge the datframes
        data = data.append(data2)

        # To drop duplicated rows
        data.drop_duplicates(inplace=True)

        # shuffle rows
        data = data.sample(frac = 1, random_state=44).reset_index(drop=True)

        nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])
        spacy_stop_words = spacy.lang.en.STOP_WORDS # getting spacy's stop-words

        #Stopwords list from https://github.com/Yoast/YoastSEO.js/blob/develop/src/config/stopwords.js
        # Convert it to a Python list and paste it here
        stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

        stop_words = list(set(list(spacy_stop_words) + stopwords))
        stopwords = list({word.lemma_.lower() for word in nlp(' '.join(stop_words))})

        # defining tokenzer function to tokenize the lower case lemma of documents in a corpus and 
        # filter out stop-words  
        def tokenizer_spacy(text):
            return [word.lemma_.lower() for word in nlp(text) if word.is_alpha and word.lemma_.lower() not in stopwords]

        #Replace the target colunms with binary number
        data.replace({'ham':1,'spam':0},inplace=True)

        sentences = data['message']
        labels=data['label']

        #Replace the target colunms with binary number
        data.replace({'ham':1,'spam':0},inplace=True)

        senten = [word for word in sentences if word not in stopwords] # stopword filtering

        # tokenize (lemmatize and filter stop words) corpus 
        senten = [' '.join(tokenizer_spacy(doc)) for doc in senten]

        # word tokenizing
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(senten)

        word_index = tokenizer.word_index

        vocab_size=len(word_index)

        # padding and converting to numeric sequence
        sequences = tokenizer.texts_to_sequences(sentences)
        padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


        from sklearn.model_selection import train_test_split
        training_sequences, test_sequences, training_labels, test_labels = train_test_split(padded, labels, test_size=0.25, random_state=42)

        from sklearn.utils import class_weight
        class_weights = dict(zip(np.unique(labels), class_weight.compute_class_weight('balanced',
                                                     np.unique(labels),
                                                     labels))) 

        # Modelling
        tf.random.set_seed(1) # for reproducibility

        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(305)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        model.summary()

        num_epochs = 12

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=5,
                                       verbose=1,
                                       min_delta=1e-4)

        lrp = tf.keraslrp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1,
                                           patience=2,
                                           cooldown=2,
                                           verbose=1)

        training_padded = np.array(training_sequences)
        training_labels = np.array(training_labels)
        testing_padded = np.array(test_sequences)
        testing_labels = np.array(test_labels)

        history = model.fit(training_padded, training_labels, epochs=num_epochs, 
                            class_weight=class_weights, 
                            validation_data=(testing_padded, testing_labels),
                            callbacks=[early_stopping, lrp],
                            verbose=2)

        print("Training Complete")

        # serialize model to JSON

        model_json = model.to_json()

        with open(f'{data_path}/model.json', 'w') as json_file:
            json_file.write(model_json)

        # serialise weights to HDF5
        model.save_weights(f'{data_path}/model.h5')

In [6]:
run = deploy(data_path)

Using TensorFlow backend.
/home/jovyan/.local/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/jovyan/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0 1], y=0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
       ..
5572    0
5573    1
5574    1
5575    1
5576    1
5577    1
5578    1
5579    1
5580    1
5581    0
5582    0
5583    1
5584    1
5585    1
5586    1
5587    0
5588    1
5589    1
5590    1
5591    1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 100)          763600    
_________________________________________________________________
dropout (Dropout)            (None, 150, 100)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 610)               990640    
_________________________________________________________________
dropout_1 (Dropout)          (None, 610)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 611       
Total params: 1,754,851
Trainable params: 1,754,851
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 4201 samples, validate on 1401 samples
Epoch 1/12
420

# Data Injestion

In [9]:
def data_injestion(data_path):
    global loaded_model
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    
    import pandas as pd
    import tensorflow
    
    data = pd.read_csv('https://raw.githubusercontent.com/Chizzy-codes/g02-sms-spam/master/data/main_data.csv', usecols=['label', 'message'],  encoding = 'latin-1')
  
    #Save the injested data as a pickle file to be used by the data tranformation component.
    with open(f'{data_path}/inj_data', 'wb') as f:
        pickle.dump(data, f)

In [10]:
idata = data_injestion(data_path)

# Data Transformation

In [17]:
def data_transformation(data_path):
    
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy==2.2.0'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==2.2.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ipython'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz'])

    
    from IPython import get_ipython
    from math import sqrt
    from numpy import concatenate
    from pandas import read_csv, DataFrame, concat
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from sklearn.metrics import mean_squared_error
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.utils import to_categorical
    from keras import regularizers

    import pandas as pd
    import numpy as np
    import pickle
    import spacy
    
    
    embedding_dim = 100
    max_length = 150
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    
    # Load and unpack the data
    
    with open(f'{data_path}/inj_data','rb') as f:
        data = pickle.load(f)
    
    nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])
    spacy_stop_words = spacy.lang.en.STOP_WORDS # getting spacy's stop-words

    #Stopwords list from https://github.com/Yoast/YoastSEO.js/blob/develop/src/config/stopwords.js
    # Convert it to a Python list and paste it here
    stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

    stop_words = list(set(list(spacy_stop_words) + stopwords))
    stopwords = list({word.lemma_.lower() for word in nlp(' '.join(stop_words))})

    # defining tokenzer function to tokenize the lower case lemma of documents in a corpus and 
    # filter out stop-words  
    def tokenizer_spacy(text):
        return [word.lemma_.lower() for word in nlp(text) if word.is_alpha and word.lemma_.lower() not in stopwords]

    #Replace the target colunms with binary number
    data.replace({'ham':1,'spam':0},inplace=True)

    sentences = data['message']
    
    try:
        labels = data['label']
    except:
        label = None
    else:
        labels = data['label']

    #Replace the target colunms with binary number
    #data.replace({'ham':1,'spam':0},inplace=True)

    senten = [word for word in sentences if word not in stopwords] # stopword filtering

    # tokenize (lemmatize and filter stop words) corpus 
    senten = [' '.join(tokenizer_spacy(doc)) for doc in senten]

    # word tokenizing
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(senten)

    word_index = tokenizer.word_index

    vocab_size=len(word_index)

    # padding and converting to numeric sequence
    sequences = tokenizer.texts_to_sequences(sentences)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


    padded = np.array(padded)
    labels = np.array(labels)

    #Save the preprocessed data as a pickle file to be used by the model component.
    with open(f'{data_path}/preprocessed_data', 'wb') as f:
        pickle.dump((padded, labels), f)

In [18]:
new_data = data_transformation(data_path)

# Model Prediction

In [19]:
def predict(data_path):
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.3.0'])
    
    import pickle
    import pandas as pd
    import numpy as np
    import tensorflow
    
    from sklearn.metrics import f1_score, precision_score
    import os
    
    with open(f'{data_path}/preprocessed_data','rb') as f:
        padded, labels = pickle.load(f)
    
    
    arc = os.path.join(os.getcwd(),'pipe_data/model.json')
    weights = os.path.join(os.getcwd(),'pipe_data/model.h5')

    # load json and create model
    #json_file = open('https://raw.githubusercontent.com/Chizzy-codes/g02-sms-spam/master/pipeline/pipe_data/model.json', 'r')
    json_file = open(arc, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = tensorflow.keras.models.model_from_json(loaded_model_json)

    # load weights into new model
    #loaded_model.load_weights('pipe_data/model.h5')
    loaded_model.load_weights(weights)

    # compile loaded model

    loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    
    
     # Load and unpack the test_data
    
    
    
    prediction = loaded_model.predict(padded)
       
    prediction = [int(np.round(i)) for i in prediction]
           
    with open(f'{data_path}/result', 'wb') as f:
        pickle.dump(prediction, f)
    
    print('Successfully!')

In [20]:
predicted = predict(data_path)

Successfully!


# Showing Result

In [23]:
def monitor(data_path):
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.24.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
    import pandas as pd
    import numpy as np
    import pickle
    
    from sklearn.metrics import f1_score, precision_score
    
    with open(f'{data_path}/preprocessed_data','rb') as f:
        padded, labels = pickle.load(f)
    
    
    with open(f'{data_path}/result', 'rb') as f:
        prediction = pickle.load(f)
        
    evaluation = []
        
    if labels.all() != None:
    
        f1 = 'F1 score: {:.4f}'.format(f1_score(labels, prediction))

        precision = 'Precision_score: {:.4f}'.format(precision_score(labels, prediction))
    
        evaluation.append(f1)
        evaluation.append(precision)
    
    with open(f'{data_path}/result.txt', 'w') as f:
        f.write(" Evaluation: {}".format(evaluation))
        f.write("\n")
        f.write(" Predictions: {}".format(prediction))
        
        
    print('Done!!!')


In [24]:
get = monitor(data_path)

Done!!!


In [25]:
# Create components.
deploy_op = comp.func_to_container_op(deploy, base_image = "tensorflow/tensorflow:latest-gpu-py3")
inj_op = comp.func_to_container_op(data_injestion , base_image = "tensorflow/tensorflow:latest-gpu-py3")
transformation_op = comp.func_to_container_op(data_transformation, base_image = "tensorflow/tensorflow:latest-gpu-py3")
predict_op = comp.func_to_container_op(predict, base_image = "tensorflow/tensorflow:latest-gpu-py3")
monitor_op = comp.func_to_container_op(monitor, base_image = "tensorflow/tensorflow:latest-gpu-py3")

# Build Kubeflow Pipelines

In [26]:
#Create a client to enable communication with the Pipelines API server.
client = kfp.Client()

In [33]:
# Define the pipeline
@dsl.pipeline(
    name='Sms Spam Classification Pipeline',
    description=
    'A machine learning pipeline that makes predictions on whether on not the sms content of a csv file is spam or not.'
)
# Define parameters to be fed into pipeline
def spam_pipeline(data_path: str):

    # Define volume to share data between components.
    vop = dsl.VolumeOp(name="create_volume",
                       resource_name="data-volume",
                       size="1Gi",
                       modes=dsl.VOLUME_MODE_RWO)
    
    # Create deploy component.
    deploy_container = deploy_op(data_path) \
                                    .add_pvolumes({data_path: vop.volume})

    # Create data injestion component.
    injestion_container = inj_op(data_path) \
                                    .add_pvolumes({data_path: deploy_container.pvolume})

    # Create data transformation component.
    transformation_container = transformation_op(data_path) \
                                    .add_pvolumes({data_path: injestion_container.pvolume})
    # Create model training component.
    predict_container = predict_op(data_path) \
                                    .add_pvolumes({data_path: transformation_container.pvolume})
    
    # Create model validation component.
    monitor_container = monitor_op(data_path) \
                                    .add_pvolumes({data_path: predict_container.pvolume})
    

    # Print the result of the prediction
    validation_result_container = dsl.ContainerOp(
        name="print_validation_result",
        image='library/bash:4.4.23',
        pvolumes={data_path: monitor_container.pvolume},
        arguments=['cat', f'{data_path}/result.txt'])

In [34]:
DATA_PATH = data_path

pipeline_func = spam_pipeline

In [35]:
experiment_name = 'spam_classification_kubeflow'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func, '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

/home/jovyan/.local/lib/python3.6/site-packages/kfp/dsl/_container_op.py:1028: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,
